In [ ]:
from os import makedirs
import pandas as pd
import plotly.express as px
from IPython.display import Image
from benchmarks.cli.data_loader import DEFAULT_DB_PATH, TBL

In [ ]:
db_path = DEFAULT_DB_PATH  # SQL db path to read epoch stats from
tbl = TBL                  # SQL table name to read epoch stats from
out_dir = None             # Directory to save plot PNG/JSON to (e.g. "m3", "azl")
host = None                # Description of host the stats were collected on, used in plot subtitle (e.g. "M3 Mac", "Amazon Linux")

In [ ]:
db_uri = f'sqlite:///{db_path}'
df = pd.read_sql_table(tbl, db_uri)
df

In [ ]:
(df.elapsed > 0).value_counts()

In [ ]:
df.n_rows.value_counts()

In [ ]:
chunk_sizes = list(map(str, sorted(df['soma_chunk_size'].unique())))
x = 'SOMA chunk size'
y = 'Samples / sec'
df[x] = pd.Categorical(df['soma_chunk_size'].apply(str), categories=chunk_sizes)
df[y] = df.n_rows / df.elapsed
df

In [ ]:
[hostname] = df.hostname.unique()
hostname

In [ ]:
[batch_size] = df.batch_size.unique()
batch_size

In [ ]:
if host is None:
    if 'm3.lan' in hostname:
        host = 'M3 Mac'
    elif 'us-west-2' in hostname:
        host = 'Amazon Linux'
host_str = f"{host}, " if host else ""

In [ ]:
dims = dict(width=1200, height=800)
grid = '#ccc'
fig = px.violin(
    df, x=x, y=y,
    category_orders={ y: chunk_sizes, },
    color='fmt',
    box=True,
    points='all',
    labels={ 'fmt': "Format", }
).update_layout(
    plot_bgcolor='white',
    **dims,
    title=dict(
        text=f'Samples/sec<br><span style="font-size:0.8em">({host_str}{batch_size} samples per PyTorch batch)',
        x=0.5,
    )
).update_traces(
    hovertemplate=None,
).update_xaxes(
    gridcolor=grid,
).update_yaxes(
    gridcolor=grid,
    zeroline=True,
    zerolinecolor=grid,
    zerolinewidth=1,
    rangemode='tozero',
)
if out_dir:
    makedirs(out_dir, exist_ok=True)
    fig.write_image(f'{out_dir}/fig.png', **dims)
    fig.write_json(f'{out_dir}/fig.json')
Image(fig.to_image(**dims))